### Assignment 1: Discrete event modelling.

**Course:**     Simulation Modeling of Financial and Economic Systems <br>
**Student:**    Danis Alukaev <br>
**Email:**      d.alukaev@innopolis.university <br>
**Group:**      B19-DS-01 <br>

**Variant:**    Task 2 <br>

On average, cars with products arrive at some base each 30 minutes. The average unloading time of one machine is 1.5 hours. Unloading is carried out by two teams of loaders. On the territory of the base, no more than 4 cars can be in line waiting for unloading. Determine the performance of the queuing system.

### Prerequisites

In [1]:
import simpy
import random 
import os
import datetime

In [2]:
ARRIVAL_PERIOD = 30
UNLOADING_TIME = 90 
LOADERS_NUMBER = 2
QUEUE_LENGTH = 4
UNTIL = 10000

In [3]:
class Logger:
    """
    The class used to log simulation progress to the file.
    By default, saves logs to directory './logs'.
    """

    def __init__(self, date, logs_dir='./logs'):
        self.logs_dir = logs_dir
        if not os.path.exists(self.logs_dir):
            os.mkdir(self.logs_dir)
        self.filename = f"Session {date}.log"
    
    def write(self, string):
        filepath = os.path.join(self.logs_dir, self.filename)
        with open(filepath, 'a') as f:
            f.write(string + '\n')

### Simulation design

In [4]:
def get_random_interval(mean):
    """
    The method returning the random integer from exponential distribution.
    Rate parameter is derived from the given average time for the action.
    :arg mean: the average time taking the action.
    :return: time interval from exponential distribution.
    """

    num_seconds = 60
    intensity = 1 / (mean * num_seconds)
    interval = int(random.expovariate(intensity) / num_seconds)
    return interval

In [5]:
class ProductBase:
    """
    Class representing the products' base involved in the task.
    It aggregates the loader resources and implements the unloading functionality.
    The time required by loaders is taken at random from exponential distribution.
    """

    def __init__(self, env, loader_number, unloading_time, queue_length):
        self.env = env
        self.loader_number = loader_number
        self.loaders = simpy.Resource(env, capacity=loader_number)
        self.unloading_time = unloading_time
        self.queue = []
        self.queue_length = queue_length
    
    def unload(self, car, logger):
        interval = get_random_interval(self.unloading_time)
        yield self.env.timeout(interval)
        # print(f"Car {car.id} was unloaded at {env.now:.2f}.")
        logger.write(f"{self.env.now}: {car.id} unloaded")

In [6]:
class Car:
    """
    Class representing the truck carrying the goods from suppliers.
    Implements arriving mechanism: if the queue is full, the car is rejected 
    by product base. Otherwise, the product base accepts the truck and performs the 
    unloading procedure.  
    """

    def __init__(self, env, id):
        self.env = env
        self.id = id
    
    def arrive(self, product_base: ProductBase, logger: Logger):
        if len(product_base.queue) > product_base.queue_length - 1:
            logger.write(f"{self.env.now}: {self.id} rejected")
            # print(f"Car {self.id} was rejected by products base.")
            return
        with product_base.loaders.request() as request:
            product_base.queue.append(self.id)
            yield request 
            product_base.queue.remove(self.id)
            logger.write(f"{self.env.now}: {self.id} accepted")
            # print(f"Car {self.id} enters the base at {self.env.now:.2f}.")
            yield env.process(product_base.unload(self, logger))
            logger.write(f"{self.env.now}: {self.id} left")
            # print(f"Car {self.id} leaves the base at {self.env.now:.2f}.")

In [7]:
def setup(env, arrival_period, unloading_time, loaders_number, queue_length, logger):
    """
    The method used to configure all the classes implemented and start the 
    request flow.
    """

    product_base = ProductBase(env, loaders_number, unloading_time, queue_length)
    cars = []

    while True:
        interval = get_random_interval(arrival_period)
        yield env.timeout(interval)
        cars.append(Car(env, len(cars)+1))
        env.process(cars[-1].arrive(product_base, logger))

In [8]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
logger = Logger(timestamp)

In [9]:
env = simpy.Environment()
args = (env, ARRIVAL_PERIOD, UNLOADING_TIME, LOADERS_NUMBER, QUEUE_LENGTH, logger)

In [10]:
env.process(setup(*args))
env.run(until=UNTIL)